<a href="https://colab.research.google.com/github/csikasote/xls-r-bem-exp/blob/main/xls_r_bemba_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install datasets transformers torchaudio librosa huggingface_hub jiwer
!pip install https://github.com/kpu/kenlm/archive/master.zip
!pip install pyctcdecode

     |████████████████████████████████| 311 kB 13.8 MB/s 
     |████████████████████████████████| 3.5 MB 87.3 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 243 kB 92.2 MB/s 
     |████████████████████████████████| 133 kB 84.1 MB/s 
     |████████████████████████████████| 1.1 MB 71.6 MB/s 
     |████████████████████████████████| 6.8 MB 76.3 MB/s 
     |████████████████████████████████| 596 kB 67.4 MB/s 
     |████████████████████████████████| 895 kB 83.7 MB/s 
     |████████████████████████████████| 50 kB 7.7 MB/s 
     |████████████████████████████████| 271 kB 71.5 MB/s 
     |████████████████████████████████| 144 kB 72.6 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     / 541 kB 2.1 MB/s
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2334664 sha256=d248620dd8227b11ef78faa56c0e23034ee851feef6aaa692a1fd6a06f492b8c
  Stored in directory: /tmp/pip-ephem-wheel-cache-6749vgpk/wheels/3d

In [4]:
!pip install bitsandbytes-cuda111

     |████████████████████████████████| 4.0 MB 16.4 MB/s 


In [47]:
%%capture
!apt install git-lfs

In [58]:
# Download BembaSpeech dataset from github repo
!git clone https://github.com/csikasote/BembaSpeech.git 
%cd BembaSpeech
!python prepare.py
%cd /content

Cloning into 'BembaSpeech'...
remote: Enumerating objects: 35625, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 35625 (delta 109), reused 7 (delta 1), pack-reused 35383
Receiving objects: 100% (35625/35625), 2.44 GiB | 25.98 MiB/s, done.
Resolving deltas: 100% (17681/17681), done.
Checking out files: 100% (15140/15140), done.
/content/BembaSpeech
Preparing audio files and generating csv files for training the model ... 

N/A% (0 of 3) |                           | Elapsed Time: 0:00:00 ETA:  --:--:--> 'train.csv' has been processed successfully!
> Training set size: 20.05 hrs
> No of files: 11892
 33% (1 of 3) |#########                  | Elapsed Time: 0:00:01 ETA:   0:00:02> 'dev.csv' has been created successfully!
> Evaluation set size: 2.42 hrs
> No of files: 1549
 66% (2 of 3) |##################         | Elapsed Time: 0:00:02 ETA:   0:00:01> 'test.csv' has been created successfully!
> Test set size: 2.05 hr

In [52]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/research_projects/robust-speech-event/run_speech_recognition_ctc_bnb.py

--2022-02-20 10:17:30--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/research_projects/robust-speech-event/run_speech_recognition_ctc_bnb.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31249 (31K) [text/plain]
Saving to: ‘run_speech_recognition_ctc_bnb.py’

run_speech_recognit 100%[===================>]  30.52K  --.-KB/s    in 0s      

2022-02-20 10:17:30 (131 MB/s) - ‘run_speech_recognition_ctc_bnb.py’ saved [31249/31249]



In [34]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [56]:
!echo '''python run_speech_recognition_ctc_bnb.py \
	--dataset_name="BembaSpeech" \
	--model_name_or_path="facebook/wav2vec2-xls-r-1b" \
	--dataset_config_name="bem" \
  --train_split_name="train" \
  --train_split_path="/content/BembaSpeech/BembaSpeech/csv/train.csv" \
  --eval_split_name="eval" \
  --eval_split_path="/content/BembaSpeech/BembaSpeech/csv/dev.csv" \
	--output_dir="./xls-r-bem-test" \
	--overwrite_output_dir \
	--max_steps="10" \
	--per_device_train_batch_size="2" \
	--learning_rate="3e-4" \
	--save_total_limit="1" \
	--evaluation_strategy="steps" \
  --audio_column_name="audio" \
	--text_column_name="sentence" \
	--length_column_name="input_length" \
	--save_steps="5" \
	--layerdrop="0.0" \
	--freeze_feature_encoder \
	--gradient_checkpointing \
	--fp16 \
	--group_by_length \
	--push_to_hub \
	--use_auth_token \
	--do_train \
  --do_eval''' > run.sh

In [ ]:
!bash run.sh

02/20/2022 10:26:22 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/20/2022 10:26:22 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=True,
greater_is_better=None,
group_by_length=True,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smo